# Performing a Threat Hunt using Python

In [128]:
import paramiko, sys, os, re
from getpass import getpass

In [129]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information to log into the remote system
host = "192.168.6.71"
port = 2222
username = "cole.davisbrand"
password = thePass

# Connect to remote system
try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication failed")

In [130]:
# Upload the Kraken binary
sftp = ssh.open_sftp()
sftp.put('kraken', "/home/cole.davisbrand/kraken")

<SFTPAttributes: [ size=9962192 uid=1286604553 gid=1286603265 mode=0o100755 atime=1667532678 mtime=1667532794 ]>

In [131]:
# Give Kraken executable perms
ssh.exec_command("chmod +x /home/cole.davisbrand/kraken")

# Create Kraken command
krakenCommand = "sudo -S /home/cole.davisbrand/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin"

# Get output from the Kraken command
stdin, stdout, stderr, = ssh.exec_command(krakenCommand)

# Must use password when running commmand with sudo privs
stdin.write(password + "\n")

# print("Kraken has finished executing")

# Create file of PIDs from Kraken output, credit Paul Gleason for the help on this!
for line in stdout.readlines():
    if re.findall(r'.*pid.*', line):
        with open('pid.txt', 'w') as f:
            f.write(line.split(" ")[-2].split("=")[-1] + ' ')


In [132]:
# Run "lsof -p" against PIDs found in Kraken output

lines = []

with open('pid.txt', 'r') as f:
    for line in f:
        lines.append(line)

for item in lines:
    splitLines = item.split(" ")

for t in splitLines:

    command = "sudo -S lsof -p " + t

    stdin, stdout, stderr = ssh.exec_command(command)
    stdin.write(thePass + "\n")
    stdin.flush()

    # Write output to a file for better visability
    for s in stdout.readlines():
        with open('lsofOutput.txt', 'a') as f:
            f.write(s)

# From here I found that the user p.dalton seemed suspicious based out the lsof -p output. Looking at the lsofOutput.txt
# file it was clear that this neeeded to be looked into further

In [120]:
# Move files from suspicious user to home directory
copyFiles = "sudo -S cp -rp /home/p.dalton /home/cole.davisbrand && sudo -S cp -rp /usr/local/bin/egrep /home/cole.davisbrand"
stdin, stdout, stderr, = ssh.exec_command(copyFiles)
# Must use password when running commmand with sudo privs
stdin.write(password + "\n")

In [133]:
# Download all suspicious files
sftp = ssh.open_sftp()

files = ['/home/cole.davisbrand/p.dalton/d.pid','/home/cole.davisbrand/p.dalton/aqwerpuqwerj','/usr/local/bin/egrep']

# Loop through the files and download them
for eachFile in files:

    # Create a list from the filename
    x = eachFile.split("/")
    # -1 is used to pluck the last element in a list
    filename = x[-1]

    # Local path to store files
    localPath = 'p.dalton/' + filename
    sftp.get(eachFile,localPath)

In [134]:
# Contents of suspicious ThugStyle username listed
stdin, stdout, stderr = ssh.exec_command("/usr/bin/ls -la /home/p.dalton")
print("".join(stdout.readlines()))


total 528
drwxr-xr-x  4 p.dalton p.dalton   4096 Oct 25 23:31 .
drwxr-xr-x 66 root     root       4096 Nov  2 19:07 ..
-rwxr-xr-x  1 p.dalton p.dalton    360 Oct 25 23:32 .bash_history
drwxr-xr-x  2 p.dalton p.dalton   4096 Feb 26  2022 .cache
-rwxr-xr-x  1 p.dalton p.dalton    850 Oct 25 23:32 .d.log
-rwxr-xr-x  1 p.dalton p.dalton 511476 Feb 18  2022 aqwerpuqwerj
-rw-r--r--  1 p.dalton p.dalton      6 Oct 25 23:32 d.pid
drwxr-xr-x  2 p.dalton p.dalton   4096 Oct 29 19:51 test-lk



What is the process name?
egrep

Which user is running the process?
p.dalton

Has the user logged in before via SSH?
yes, 4 times

When did the user login?
Oct 25, 2022 at 23:23:31

How long was the login session?
23:23:31 - 23:31:58 (9 minutes)

What is located in the unauthorized user’s home directory?
3 files: aqwerpuqwerj, d.pid, test-lk

Reflections:

Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed? p.dalton follows the trend of other ThugStyle threat actors. This is based on the naming scheme, SSH as the method of authentication, and storing files a bin directory.

What did you like the most and least about this assignment? I liked how the we were given a threat report and then had the chance to use it to solve the puzzle. I wish we had a bit more guidance with how to navigate this assignment.

What additional questions do you have? None at the moment
